xgboosting, gradient boosting, ensangble 모델 비교

In [39]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error


In [40]:
def convert_time_to_minutes(time_str): 
    hours, minutes = time_str.split('h')
    hours = float(hours.strip()) 
    minutes = minutes.replace('m', '').strip()
    minutes = int(minutes) if minutes else 0
    total_minutes = int(round(hours * 60)) + minutes
    return total_minutes

def time_of_day(hour):
    if 5 <= hour < 12:
        return 'Morning'
    elif 12 <= hour < 17:
        return 'Afternoon'
    elif 17 <= hour < 21:
        return 'Evening'
    else:
        return 'Night'
    
reference_date = pd.to_datetime('2022-02-11')

In [41]:
train_data = pd.read_csv('./gist-mldl24f-hw3/train.csv')

train_data['date'] = pd.to_datetime(train_data['date'])
train_data['num_code'] = train_data['num_code'].astype(str)
train_data['time_taken'] = train_data['time_taken'].apply(convert_time_to_minutes)
train_data['dep_time'] = pd.to_datetime(train_data['dep_time'], format='%H:%M')
train_data['arr_time'] = pd.to_datetime(train_data['arr_time'], format='%H:%M')
train_data['dep_time_period'] = train_data['dep_time'].dt.hour.apply(time_of_day)
train_data['arr_time_period'] = train_data['arr_time'].dt.hour.apply(time_of_day)
train_data['stop_num'] = train_data['stop'].str.split('-').str[0].map({
    'non': 0,
    '1': 1,
    '2+': 2
})
train_data['days_since'] = (pd.to_datetime(train_data['date']) - reference_date).dt.days
train_data['day_of_week'] = train_data['date'].dt.day_name()
train_data['hour_dep'] = train_data['dep_time'].dt.hour
train_data['hour_arr'] = train_data['arr_time'].dt.hour


In [47]:
test_data = pd.read_csv('./gist-mldl24f-hw3/test.csv')
test_data['date'] = pd.to_datetime(test_data['date'])
test_data['num_code'] = test_data['num_code'].astype(str)
test_data['time_taken'] = test_data['time_taken'].apply(convert_time_to_minutes)
test_data['dep_time'] = pd.to_datetime(test_data['dep_time'], format='%H:%M')
test_data['arr_time'] = pd.to_datetime(test_data['arr_time'], format='%H:%M')
test_data['dep_time_period'] = test_data['dep_time'].dt.hour.apply(time_of_day)
test_data['arr_time_period'] = test_data['arr_time'].dt.hour.apply(time_of_day)
test_data['stop_num'] = test_data['stop'].str.split('-').str[0].map({
    'non': 0,
    '1': 1,
    '2+': 2
})
test_data['days_since'] = (pd.to_datetime(test_data['date']) - reference_date).dt.days
test_data['day_of_week'] = test_data['date'].dt.day_name()
test_data['hour_dep'] = test_data['dep_time'].dt.hour
test_data['hour_arr'] = test_data['arr_time'].dt.hour

In [43]:
cat_predictors = ['airline', 'from', 'to', 'class', 'dep_time_period', 'arr_time_period', 'day_of_week']
num_predictors = ['time_taken', 'days_since','stop_num', 'hour_dep', 'hour_arr']


In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error
import pandas as pd

# 카테고리형 변수와 수치형 변수에 대한 One-Hot Encoding 및 데이터 준비
X_cat_train = pd.get_dummies(train_data[cat_predictors], drop_first=True)
X_num_train = train_data[num_predictors]
X_train = pd.concat([X_num_train, X_cat_train], axis=1)
y_train = train_data['price']

# 테스트 데이터에 대해 동일하게 처리
X_cat_test = pd.get_dummies(test_data[cat_predictors], drop_first=True)
X_num_test = test_data[num_predictors]
X_test = pd.concat([X_num_test, X_cat_test], axis=1)
X_test = X_test.reindex(columns=X_train.columns, fill_value=0)

# 1. Gradient Boosting 모델
gb_model = GradientBoostingRegressor(n_estimators=100, max_depth=10, random_state=0)
gb_model.fit(X_train, y_train)
y_pred_gb = gb_model.predict(X_test)
gb_mae = mean_absolute_error(y_train, gb_model.predict(X_train))  # 평가 지표 예시

# 2. XGBoost 모델
xgb_model = XGBRegressor(n_estimators=100, max_depth=10, random_state=0)
xgb_model.fit(X_train, y_train)
y_pred_xgb = xgb_model.predict(X_test)
xgb_mae = mean_absolute_error(y_train, xgb_model.predict(X_train))  # 평가 지표 예시

# 3. 앙상블 모델 (Gradient Boosting + XGBoost의 예측 평균)
y_pred_ensemble = (y_pred_gb + y_pred_xgb) / 2
ensemble_mae = mean_absolute_error(y_train, (gb_model.predict(X_train) + xgb_model.predict(X_train)) / 2)

print(gb_mae,xgb_mae,ensemble_mae)

best_model = min([(y_pred_gb, gb_mae), (y_pred_xgb, xgb_mae), (y_pred_ensemble, ensemble_mae)], key=lambda x: x[1])
y_pred_best = best_model[0]

# 결과 DataFrame 생성 및 CSV 파일로 저장
result = pd.DataFrame({'id': test_data['id'], 'price': y_pred_best})
result.to_csv('best_model_pred.csv', index=False)
print("best_model_pred.csv 파일이 생성되었습니다.")


1376.466003304934 917.8535481792303 1092.8635966811619
best_model_pred.csv 파일이 생성되었습니다.


In [45]:
print(train_data.isna().sum())


id                 0
date               0
airline            0
ch_code            0
num_code           0
dep_time           0
from               0
time_taken         0
stop               0
arr_time           0
to                 0
class              0
price              0
dep_time_period    0
arr_time_period    0
stop_num           0
days_since         0
day_of_week        0
hour_dep           0
hour_arr           0
dtype: int64


In [46]:
print(test_data.isna().sum())


id                    0
date                  0
airline               0
ch_code               0
num_code              0
dep_time              0
from                  0
time_taken            0
stop                  0
arr_time              0
to                    0
class                 0
dep_time_period       0
arr_time_period       0
stop_num           1315
days_since            0
day_of_week           0
hour_dep              0
hour_arr              0
dtype: int64
